In [1]:
from agential.benchmarks.computer_use.webvoyager.data_manager import WebVoyagerDataManager

In [2]:
meta = WebVoyagerDataManager()

In [5]:
meta.data[0]

{'web_name': 'Allrecipes',
 'id': 'Allrecipes--0',
 'ques': 'Provide a recipe for vegetarian lasagna with more than 100 reviews and a rating of at least 4.5 stars suitable for 6 people.',
 'web': 'https://www.allrecipes.com/'}

# test action format

In [2]:
import re

def extract_information(text):
    patterns = {
        "click": r"Click \[?(\d+)\]?",
        "type": r"Type \[?(\d+)\]?[; ]+\[?(.[^\]]*)\]?",
        # "delete_and_type": r"Delete_and_Type \[?(\d+)\]?[; ]+\[?(.[^\]]*)\]?",
        "scroll": r"Scroll \[?(\d+|WINDOW)\]?[; ]+\[?(up|down)\]?",
        "wait": r"^Wait",
        "goback": r"^GoBack",
        "google": r"^Google",
        "answer": r"ANSWER[; ]+\[?(.[^\]]*)\]?",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            if key in ["click", "wait", "goback", "google"]:
                # no content
                return key, match.groups()
            else:
                return key, (
                    {"number": match.group(1), "content": match.group(2)}
                    if key in ["type", "scroll"]
                    else {"content": match.group(1)}
                )
    return None, None

text = "Click [123]"
output = extract_information(text)
print(output)

('click', ('123',))


In [12]:
pattern = r"Click \[?(\d+)\]?"
match = re.search(pattern, "Click [123]")
match.groups()

('123',)

In [3]:
text = "Type [45]; [Hello World]"
output = extract_information(text)
print(output)
# Output: ('type', {'number': '45', 'content': 'Hello World'})


('type', {'number': '45', 'content': 'Hello World'})


In [9]:
text = "Scroll [3]; [down]"
output = extract_information(text)
print(output)
# Output: ('scroll', {'number': '3', 'content': 'down'})

text = "Scroll [WINDOW]; [up]"
output = extract_information(text)
print(output)
# Output: ('scroll', {'number': 'WINDOW', 'content': 'up'})


('scroll', {'number': '3', 'content': 'down'})
('scroll', {'number': 'WINDOW', 'content': 'up'})


In [4]:
text = "Wait for the next step"
output = extract_information(text)
print(output)
# Output: ('wait', ())


('wait', ())


In [5]:
text = "GoBack to the previous page"
output = extract_information(text)
print(output)
# Output: ('goback', ())


('goback', ())


In [6]:
text = "Google search for cats"
output = extract_information(text)
print(output)
# Output: ('google', ())

('google', ())


In [7]:
text = "ANSWER; [This is the answer]"
output = extract_information(text)
print(output)
# Output: ('answer', {'content': 'This is the answer'})

('answer', {'content': 'This is the answer'})


In [8]:
text = "Unknown command with random text"
output = extract_information(text)
print(output)
# Output: (None, None)

(None, None)


# test pdf

In [13]:
from pathlib import Path
pdf_path = "test.pdf"
pdf_content = Path(pdf_path).read_bytes()


In [15]:
type(pdf_content)

bytes

In [22]:
import litellm
litellm.utils.supports_pdf_input(model="gpt-4")

False

In [ ]:
task="Click on the data folder"


prompt = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant that can analyze the content of a PDF file. "
            "You will be provided with the content of the PDF and a task. "
            "Please analyze the content and provide an answer that matches the task."
        ),
    },
    {"role": "user", "content": f"Task: {task}\nPDF Content: {pdf_content}"},
]
llm = LLM(model="gpt-4o")
response = llm(prompt)

In [16]:
from agential.core.llm import LLM


def get_pdf_retrieval_ans_from_llm(llm, pdf_path: str, task: str) -> str:
    """
    Retrieve an answer to a task using the LLM class with a PDF file.

    Args:
        llm (LLM): An instance of the LLM class for generating responses.
        pdf_path (str): The path to the PDF file to analyze.
        task (str): The task or question to be answered based on the PDF content.

    Returns:
        str: The assistant's response based on the PDF content and task.
    """
    pdf_content = Path(pdf_path).read_bytes()

    prompt = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that can analyze the content of a PDF file. "
                "You will be provided with the content of the PDF and a task. "
                "Please analyze the content and provide an answer that matches the task."
            ),
        },
        {"role": "user", "content": f"Task: {task}\nPDF Content: {pdf_content}"},
    ]

    response = llm(prompt)

    return response

response = get_pdf_retrieval_ans_from_llm(LLM("gpt-4o"), "test.pdf", task="Click on the data folder")


c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



ContextWindowExceededError: litellm.ContextWindowExceededError: litellm.BadRequestError: ContextWindowExceededError: OpenAIException - Error code: 400 - {'error': {'message': "Invalid 'messages[1].content': string too long. Expected a string with maximum length 1048576, but got a string with length 13011025 instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content', 'code': 'string_above_max_length'}}